# GNN con Demográficos + Biomarcadores CSF
Este notebook extiende DemographicOnly.ipynb añadiendo biomarcadores de líquido cefalorraquídeo (CSF):
- ABETA42 (Amiloide beta 42)
- TAU (Proteína tau total)
- PTAU (Proteína tau fosforilada)

Objetivo: Predecir años hasta el inicio de síntomas de Alzheimer usando GNN

In [30]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

torch.manual_seed(42)
np.random.seed(42)

In [31]:
csv_path = "./data/adni/demographics/PTDEMOG.csv"
df = pd.read_csv(csv_path)
print(f"Demographics loaded: {df.shape}")
print(f"Columns: {list(df.columns)}")

Demographics loaded: (6210, 84)
Columns: ['PHASE', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'PTSOURCE', 'PTGENDER', 'PTDOB', 'PTDOBYY', 'PTHAND', 'PTMARRY', 'PTEDUCAT', 'PTWORKHS', 'PTWORK', 'PTNOTRT', 'PTRTYR', 'PTHOME', 'PTTLANG', 'PTPLANG', 'PTADBEG', 'PTCOGBEG', 'PTADDX', 'PTETHCAT', 'PTRACCAT', 'PTIDENT', 'PTORIENT', 'PTORIENTOT', 'PTENGSPK', 'PTNLANG', 'PTENGSPKAGE', 'PTCLANG', 'PTLANGSP', 'PTLANGWR', 'PTSPTIM', 'PTSPOTTIM', 'PTLANGPR1', 'PTLANGSP1', 'PTLANGRD1', 'PTLANGWR1', 'PTLANGUN1', 'PTLANGPR2', 'PTLANGSP2', 'PTLANGRD2', 'PTLANGWR2', 'PTLANGUN2', 'PTLANGPR3', 'PTLANGSP3', 'PTLANGRD3', 'PTLANGWR3', 'PTLANGUN3', 'PTLANGPR4', 'PTLANGSP4', 'PTLANGRD4', 'PTLANGWR4', 'PTLANGUN4', 'PTLANGPR5', 'PTLANGSP5', 'PTLANGRD5', 'PTLANGWR5', 'PTLANGUN5', 'PTLANGPR6', 'PTLANGSP6', 'PTLANGRD6', 'PTLANGWR6', 'PTLANGUN6', 'PTLANGTTL', 'PTETHCATH', 'PTASIAN', 'PTOPI', 'PTBORN', 'PTBIRPL', 'PTIMMAGE', 'PTIMMWHY', 'PTBIRPR', 'PTBIRGR', 'ID', 'SITEID', 'USERDATE', 'USERDATE2', 'DD_CRF_VERSI

In [32]:
biomarker_path = "./data/adni/demographics/UPENNBIOMK_ROCHE_ELECSYS_11Oct2025.csv"
df_bio = pd.read_csv(biomarker_path)
print(f"\nBiomarkers loaded: {df_bio.shape}")
print(f"Columns: {list(df_bio.columns)}")
print(f"\nMissing values in biomarkers:")
print(df_bio[['ABETA40', 'ABETA42', 'TAU', 'PTAU']].isnull().sum())

bio_cols_to_use = ['RID', 'VISCODE2', 'EXAMDATE', 'ABETA42', 'TAU', 'PTAU']
df_bio = df_bio[bio_cols_to_use].copy()

for col in ['ABETA42', 'TAU', 'PTAU']:
    df_bio[col] = pd.to_numeric(df_bio[col], errors='coerce')

print(f"\nBiomarker stats:")
print(df_bio[['ABETA42', 'TAU', 'PTAU']].describe())


Biomarkers loaded: (3174, 13)
Columns: ['PHASE', 'PTID', 'RID', 'VISCODE2', 'EXAMDATE', 'BATCH', 'RUNDATE', 'ABETA40', 'ABETA42', 'TAU', 'PTAU', 'COMMENT', 'update_stamp']

Missing values in biomarkers:
ABETA40    2240
ABETA42       7
TAU          15
PTAU         27
dtype: int64

Biomarker stats:
           ABETA42          TAU         PTAU
count  3167.000000  3159.000000  3147.000000
mean   1062.012409   286.722605    27.273298
std     620.162858   128.400411    14.270766
min     203.000000    80.080000     8.000000
25%     592.650000   195.700000    17.175000
50%     870.400000   256.900000    23.540000
75%    1423.500000   344.150000    33.455000
max    4779.000000  1018.000000   108.500000


In [33]:
def norm_codes_to_labels(s: pd.Series, mapping: dict) -> pd.Series:
    out = s.astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
    out = out.map(mapping)
    return out

gender_map = {"1":"male","2":"female","male":"male","female":"female","m":"male","f":"female"}
marry_map  = {"1":"married","2":"widowed","3":"divorced","4":"never_married","6":"domestic_partnership"}

def to_year(s):
    s = pd.to_numeric(s, errors="coerce")
    s = s.where((s >= 1900) & (s <= 2100))
    return s

In [34]:

onset_cols = [c for c in ["PTCOGBEG","PTADBEG","PTADDX"] if c in df.columns]
for c in onset_cols:
    df[c] = to_year(df[c])

def row_min_nonnull(row):
    vals = [row[c] for c in onset_cols if pd.notna(row[c])]
    return min(vals) if vals else np.nan

df["YEAR_ONSET"] = df.apply(row_min_nonnull, axis=1) if onset_cols else np.nan
df["YEAR_ONSET"] = to_year(df["YEAR_ONSET"])

for c in ["PTDOBYY","PTEDUCAT"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

if "PTGENDER" in df.columns:
    df["PTGENDER"] = norm_codes_to_labels(df["PTGENDER"], gender_map)
if "PTMARRY" in df.columns:
    df["PTMARRY"]  = norm_codes_to_labels(df["PTMARRY"], marry_map)

print(f"\nDemographics processed. Shape: {df.shape}")


Demographics processed. Shape: (6210, 85)


In [35]:

date_col = "EXAMDATE" if "EXAMDATE" in df.columns else ("VISDATE" if "VISDATE" in df.columns else None)
if not date_col:
    raise ValueError("No se encuentra columna de fecha (VISDATE/EXAMDATE)")

df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df_bio['EXAMDATE'] = pd.to_datetime(df_bio['EXAMDATE'], errors='coerce')


if 'VISCODE2' in df.columns:
    viscode_col = 'VISCODE2'
elif 'VISCODE' in df.columns:
    viscode_col = 'VISCODE'
else:
    viscode_col = None

if viscode_col:
    print(f"Merging using RID + {viscode_col}")
    
    df['VISCODE_NORMALIZED'] = df[viscode_col].astype(str).str.strip()
    df['VISCODE_NORMALIZED'] = df['VISCODE_NORMALIZED'].replace({'sc': 'bl', 'f': 'bl', 'nan': ''})
    
    df_bio['VISCODE_NORMALIZED'] = df_bio['VISCODE2'].astype(str).str.strip()
    
    print(f"\nVISCODE mapping applied:")
    ptdemog_unique = df[viscode_col].dropna().unique()
    ptdemog_norm_unique = df['VISCODE_NORMALIZED'].unique()
    upenn_unique = df_bio['VISCODE_NORMALIZED'].unique()
    
    print(f"  PTDEMOG unique values (original): {sorted([str(x) for x in ptdemog_unique])[:10]}")
    print(f"  PTDEMOG unique values (normalized): {sorted([str(x) for x in ptdemog_norm_unique])[:10]}")
    print(f"  UPENNBIOMK unique values: {sorted([str(x) for x in upenn_unique])[:10]}")
    
    df_merged = df.merge(
        df_bio[['RID', 'VISCODE_NORMALIZED', 'ABETA42', 'TAU', 'PTAU']], 
        on=['RID', 'VISCODE_NORMALIZED'],
        how='left',
        suffixes=('', '_bio')
    )
    
    df_merged = df_merged.drop(columns=['VISCODE_NORMALIZED'])
    
else:
    print("VISCODE not found. Merging using RID + closest date match")
    df_merged = pd.merge_asof(
        df.sort_values([date_col]),
        df_bio[['RID', 'EXAMDATE', 'ABETA42', 'TAU', 'PTAU']].sort_values('EXAMDATE'),
        left_on=date_col,
        right_on='EXAMDATE',
        by='RID',
        direction='nearest',
        tolerance=pd.Timedelta('180 days')  # máximo 6 meses de diferencia
    )

df = df_merged

print(f"\nAfter merge: {df.shape}")
print(f"Rows with biomarker data: {df[['ABETA42', 'TAU', 'PTAU']].notna().any(axis=1).sum()}")
print(f"\nMissing biomarkers after merge:")
print(df[['ABETA42', 'TAU', 'PTAU']].isnull().sum())

print(f"\nMerge quality check:")
print(f"  % of rows with at least one biomarker: {df[['ABETA42', 'TAU', 'PTAU']].notna().any(axis=1).sum() / len(df) * 100:.1f}%")

Merging using RID + VISCODE2

VISCODE mapping applied:
  PTDEMOG unique values (original): ['bl', 'f', 'm03', 'm06', 'm12', 'm126', 'm132', 'm138', 'm144', 'm150']
  PTDEMOG unique values (normalized): ['', 'bl', 'm03', 'm06', 'm12', 'm126', 'm132', 'm138', 'm144', 'm150']
  UPENNBIOMK unique values: ['bl', 'm102', 'm108', 'm114', 'm12', 'm120', 'm126', 'm132', 'm138', 'm144']

After merge: (6210, 88)
Rows with biomarker data: 1793

Missing biomarkers after merge:
ABETA42    4419
TAU        4424
PTAU       4428
dtype: int64

Merge quality check:
  % of rows with at least one biomarker: 28.9%


In [36]:


df['TAU_ABETA42_RATIO'] = df['TAU'] / (df['ABETA42'] + 1e-6)
df['PTAU_ABETA42_RATIO'] = df['PTAU'] / (df['ABETA42'] + 1e-6)
df['PTAU_TAU_RATIO'] = df['PTAU'] / (df['TAU'] + 1e-6)

print("Feature engineering completed.")
print(f"New ratio features: TAU_ABETA42_RATIO, PTAU_ABETA42_RATIO, PTAU_TAU_RATIO")

Feature engineering completed.
New ratio features: TAU_ABETA42_RATIO, PTAU_ABETA42_RATIO, PTAU_TAU_RATIO


In [37]:

print("="*70)
print("VALIDACIÓN DEL MERGE - Verificando relación paciente-biomarcador")
print("="*70)

sample_rids = df[df[['ABETA42', 'TAU', 'PTAU']].notna().all(axis=1)]['RID'].unique()[:3]

print(f"\n📊 Muestra de {len(sample_rids)} pacientes con biomarcadores completos:\n")

for rid in sample_rids:
    patient_data = df[df['RID'] == rid][['RID', 'PTID', viscode_col if viscode_col else 'VISDATE', 
                                         'ABETA42', 'TAU', 'PTAU']].head(3)
    print(f"Paciente RID={rid}:")
    print(patient_data.to_string(index=False))
    print()

print("\n" + "="*70)
print("📈 ESTADÍSTICAS DEL MERGE:")
print("="*70)

total_rows = len(df)
rows_with_biomarkers = df[['ABETA42', 'TAU', 'PTAU']].notna().any(axis=1).sum()
rows_complete_biomarkers = df[['ABETA42', 'TAU', 'PTAU']].notna().all(axis=1).sum()

print(f"Total de visitas: {total_rows}")
print(f"Visitas con al menos un biomarcador: {rows_with_biomarkers} ({rows_with_biomarkers/total_rows*100:.1f}%)")
print(f"Visitas con todos los biomarcadores: {rows_complete_biomarkers} ({rows_complete_biomarkers/total_rows*100:.1f}%)")

patients_with_bio = df[df[['ABETA42', 'TAU', 'PTAU']].notna().any(axis=1)]['RID'].nunique()
total_patients = df['RID'].nunique()
print(f"\nPacientes únicos (RID) con biomarcadores: {patients_with_bio}/{total_patients} ({patients_with_bio/total_patients*100:.1f}%)")

print("\n✅ Merge completado. Revisa los ejemplos arriba para verificar la coherencia.")

VALIDACIÓN DEL MERGE - Verificando relación paciente-biomarcador

📊 Muestra de 3 pacientes con biomarcadores completos:

Paciente RID=3:
 RID       PTID VISCODE2  ABETA42   TAU  PTAU
   3 011_S_0003       sc    741.5 239.7 22.83

Paciente RID=4:
 RID       PTID VISCODE2  ABETA42   TAU  PTAU
   4 022_S_0004       sc   1501.0 153.1 13.29

Paciente RID=5:
 RID       PTID VISCODE2  ABETA42   TAU  PTAU
   5 011_S_0005       sc    547.3 337.0 33.43


📈 ESTADÍSTICAS DEL MERGE:
Total de visitas: 6210
Visitas con al menos un biomarcador: 1793 (28.9%)
Visitas con todos los biomarcadores: 1780 (28.7%)

Pacientes únicos (RID) con biomarcadores: 1633/4945 (33.0%)

✅ Merge completado. Revisa los ejemplos arriba para verificar la coherencia.


In [38]:

df["EXAM_YEAR"] = to_year(df[date_col].dt.year)

df["AGE_AT_VISIT"] = np.where(
    df["EXAM_YEAR"].notna() & df["PTDOBYY"].notna(),
    df["EXAM_YEAR"] - df["PTDOBYY"],
    np.nan
)

df["YEARS_TO_ONSET"] = np.where(
    df["YEAR_ONSET"].notna() & df["EXAM_YEAR"].notna(),
    df["YEAR_ONSET"] - df["EXAM_YEAR"],
    np.nan
)

df.loc[(df["YEARS_TO_ONSET"] < 0) & df["YEAR_ONSET"].notna(), "YEARS_TO_ONSET"] = np.nan
df.loc[df["YEARS_TO_ONSET"] > 50, "YEARS_TO_ONSET"] = np.nan

df["HAS_LABEL"] = df["YEARS_TO_ONSET"].notna()

print(f"\nVisits prepared: {len(df)}")
print(f"Labeled visits (with YEARS_TO_ONSET): {df['HAS_LABEL'].sum()}")


Visits prepared: 6210
Labeled visits (with YEARS_TO_ONSET): 82


In [39]:

biomarker_cols = ['ABETA42', 'TAU', 'PTAU', 'TAU_ABETA42_RATIO', 'PTAU_ABETA42_RATIO', 'PTAU_TAU_RATIO']

df = df.sort_values(['RID', date_col])

print(f"\nBefore imputation:")
print(df[biomarker_cols].isnull().sum())

for col in biomarker_cols:
    df[col] = df.groupby('RID')[col].ffill()
    df[col] = df.groupby('RID')[col].bfill()

for col in biomarker_cols:
    if df[col].isnull().any():
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)
        print(f"{col}: filled remaining NaN with median {median_val:.3f}")

print(f"\nAfter imputation:")
print(df[biomarker_cols].isnull().sum())


Before imputation:
ABETA42               4419
TAU                   4424
PTAU                  4428
TAU_ABETA42_RATIO     4426
PTAU_ABETA42_RATIO    4430
PTAU_TAU_RATIO        4428
dtype: int64
ABETA42: filled remaining NaN with median 942.500
TAU: filled remaining NaN with median 244.800
PTAU: filled remaining NaN with median 21.990
TAU_ABETA42_RATIO: filled remaining NaN with median 0.226
PTAU_ABETA42_RATIO: filled remaining NaN with median 0.021
PTAU_TAU_RATIO: filled remaining NaN with median 0.091

After imputation:
ABETA42               0
TAU                   0
PTAU                  0
TAU_ABETA42_RATIO     0
PTAU_ABETA42_RATIO    0
PTAU_TAU_RATIO        0
dtype: int64


In [40]:

num_cols = [c for c in ["AGE_AT_VISIT", "PTEDUCAT"] + biomarker_cols if c in df.columns]

cat_cols = [c for c in ["PTGENDER", "PTMARRY"] if c in df.columns]

for c in ["AGE_AT_VISIT", "PTEDUCAT"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(df[c].median())

parts = []

if num_cols:
    scaler = StandardScaler()
    X_num = pd.DataFrame(
        scaler.fit_transform(df[num_cols]),
        columns=num_cols,
        index=df.index
    )
    parts.append(X_num)
    print(f"Numeric features ({len(num_cols)}): {num_cols}")

if cat_cols:
    X_cat = pd.get_dummies(df[cat_cols], prefix=cat_cols, drop_first=False, dtype=float)
    parts.append(X_cat)
    print(f"Categorical features ({len(cat_cols)}): {cat_cols} -> {X_cat.shape[1]} one-hot columns")

if not parts:
    raise ValueError("No features available")

X = pd.concat(parts, axis=1).astype(float)
X_clean = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)

print(f"\nFinal feature matrix shape: {X_clean.shape}")
print(f"Total features: {X_clean.shape[1]}")

Numeric features (8): ['AGE_AT_VISIT', 'PTEDUCAT', 'ABETA42', 'TAU', 'PTAU', 'TAU_ABETA42_RATIO', 'PTAU_ABETA42_RATIO', 'PTAU_TAU_RATIO']
Categorical features (2): ['PTGENDER', 'PTMARRY'] -> 7 one-hot columns

Final feature matrix shape: (6210, 15)
Total features: 15


In [41]:

n_samples = X_clean.shape[0]
k = min(8, max(1, n_samples - 1))
n_neighbors = min(n_samples, k + 1)

if n_samples >= 2:
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric="euclidean")
    nbrs.fit(X_clean.values)
    _, idx = nbrs.kneighbors(X_clean.values)
    src_knn, dst_knn = [], []
    for i in range(idx.shape[0]):
        for j in idx[i, 1:]:
            src_knn.append(i)
            dst_knn.append(j)
    edge_knn = torch.tensor([src_knn, dst_knn], dtype=torch.long)
else:
    edge_knn = torch.empty((2, 0), dtype=torch.long)

tmp = df.reset_index()[["index", "RID", date_col]].dropna(subset=[date_col]).sort_values(["RID", date_col])
src_tmp, dst_tmp = [], []
for rid, g in tmp.groupby("RID"):
    ids = g["index"].tolist()
    for a, b in zip(ids[:-1], ids[1:]):
        src_tmp.append(a)
        dst_tmp.append(b)
edge_tmp = torch.tensor([src_tmp, dst_tmp], dtype=torch.long) if src_tmp else torch.empty((2,0), dtype=torch.long)

def undirected(e):
    return torch.cat([e, e.flip(0)], dim=1) if e.numel() else e

edges = []
if edge_knn.numel(): edges.append(undirected(edge_knn))
if edge_tmp.numel(): edges.append(undirected(edge_tmp))
edge_index = torch.cat(edges, dim=1) if edges else torch.empty((2,0), dtype=torch.long)
if edge_index.numel():
    edge_index = torch.unique(edge_index, dim=1)

print(f"\nGraph constructed:")
print(f"  Nodes: {len(df)}")
print(f"  kNN edges: {edge_knn.size(1)}")
print(f"  Temporal edges: {edge_tmp.size(1)}")
print(f"  Total edges (undirected, unique): {edge_index.size(1)}")


Graph constructed:
  Nodes: 6210
  kNN edges: 49680
  Temporal edges: 1264
  Total edges (undirected, unique): 70699


In [42]:

df["USE_FOR_LABEL"] = False
if df["HAS_LABEL"].any():
    idx_last_pre = df.loc[df["HAS_LABEL"]].groupby("RID")[date_col].idxmax()
    df.loc[idx_last_pre, "USE_FOR_LABEL"] = True

rids_with_label = df.loc[df["USE_FOR_LABEL"], "RID"].dropna().unique()
rng = np.random.default_rng(42)
rng.shuffle(rids_with_label)
n_lab_rids = len(rids_with_label)

tr_n = max(1, int(0.7 * n_lab_rids))
va_n = max(0, int(0.15 * n_lab_rids))
if tr_n + va_n > max(0, n_lab_rids - 1):
    va_n = max(0, n_lab_rids - 1 - tr_n)

train_rids = set(rids_with_label[:tr_n])
val_rids   = set(rids_with_label[tr_n:tr_n+va_n])
test_rids  = set(rids_with_label[tr_n+va_n:])

node_split = np.full(len(df), "train", dtype=object)
node_rids = df["RID"].to_numpy()
node_split[np.isin(node_rids, list(val_rids))]  = "val"
node_split[np.isin(node_rids, list(test_rids))] = "test"

use_for_label = df["USE_FOR_LABEL"].to_numpy()
train_mask_np = (node_split == "train") & use_for_label
val_mask_np   = (node_split == "val")   & use_for_label
test_mask_np  = (node_split == "test")  & use_for_label

split_map = {"train":0, "val":1, "test":2}
split_idx = np.vectorize(split_map.get)(node_split)
src_np = edge_index[0].cpu().numpy()
dst_np = edge_index[1].cpu().numpy()
keep_edges = split_idx[src_np] == split_idx[dst_np]
edge_index = edge_index[:, torch.tensor(keep_edges)]

print(f"\nSplits created:")
print(f"  RIDs with labels: {n_lab_rids}")
print(f"  Train RIDs: {len(train_rids)}")
print(f"  Val RIDs: {len(val_rids)}")
print(f"  Test RIDs: {len(test_rids)}")
print(f"\n  Labeled nodes for training:")
print(f"    Train: {train_mask_np.sum()}")
print(f"    Val: {val_mask_np.sum()}")
print(f"    Test: {test_mask_np.sum()}")
print(f"\n  Edges (intra-split only): {edge_index.size(1)}")


Splits created:
  RIDs with labels: 82
  Train RIDs: 57
  Val RIDs: 12
  Test RIDs: 13

  Labeled nodes for training:
    Train: 57
    Val: 12
    Test: 13

  Edges (intra-split only): 70041


In [43]:

y_full = df["YEARS_TO_ONSET"].astype(float)
y_mu  = float(y_full[train_mask_np].mean()) if train_mask_np.any() else 0.0
y_std = float(y_full[train_mask_np].std(ddof=0)) if train_mask_np.any() else 1.0
if not np.isfinite(y_std) or y_std == 0.0:
    y_std = 1.0

y_scaled = (y_full - y_mu) / y_std
y_t = torch.tensor(y_scaled.fillna(0).values, dtype=torch.float32)

x = torch.tensor(X_clean.values, dtype=torch.float32)
train_mask = torch.tensor(train_mask_np, dtype=torch.bool)
val_mask   = torch.tensor(val_mask_np,   dtype=torch.bool)
test_mask  = torch.tensor(test_mask_np,  dtype=torch.bool)

data = Data(x=x, edge_index=edge_index, y=y_t,
            train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

print(f"\nData object created:")
print(f"  x.shape: {data.x.shape}")
print(f"  edge_index.shape: {data.edge_index.shape}")
print(f"  y.shape: {data.y.shape}")
print(f"  Target scaling: mean={y_mu:.3f}, std={y_std:.3f}")

Y_MEAN_TRAIN, Y_STD_TRAIN = y_mu, y_std


Data object created:
  x.shape: torch.Size([6210, 15])
  edge_index.shape: torch.Size([2, 70041])
  y.shape: torch.Size([6210])
  Target scaling: mean=0.018, std=0.131


In [44]:

class GNNRegressor(nn.Module):
    def __init__(self, in_ch, hid=64, dropout=0.3):
        super().__init__()
        self.c1 = GCNConv(in_ch, hid)
        self.c2 = GCNConv(hid, 1)
        self.dropout = dropout
        
    def forward(self, x, edge_index):
        x = F.relu(self.c1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.c2(x, edge_index)
        return x.squeeze(-1)

device = torch.device("cpu")
model = GNNRegressor(in_ch=data.num_node_features, hid=64, dropout=0.3).to(device)
data = data.to(device)

print(f"\nModel created:")
print(f"  Input features: {data.num_node_features}")
print(f"  Hidden dim: 64")
print(f"  Architecture: GCNConv -> ReLU -> Dropout -> GCNConv -> Output")
print(f"\nModel parameters: {sum(p.numel() for p in model.parameters())}")


Model created:
  Input features: 15
  Hidden dim: 64
  Architecture: GCNConv -> ReLU -> Dropout -> GCNConv -> Output

Model parameters: 1089


In [45]:

opt = torch.optim.AdamW(model.parameters(), lr=1e-2, weight_decay=1e-4)
loss_fn = nn.MSELoss()

def eval_metrics(split="val"):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        mask = data.train_mask if split=="train" else (data.val_mask if split=="val" else data.test_mask)
        if not mask.any():
            return float("nan"), float("nan")
        mae_scaled = torch.mean(torch.abs(out[mask] - data.y[mask])).item()
        rmse_scaled = torch.sqrt(loss_fn(out[mask], data.y[mask])).item()
        return mae_scaled * Y_STD_TRAIN, rmse_scaled * Y_STD_TRAIN  # en años

print("\nStarting training...\n")

for ep in range(1, 101):
    model.train()
    opt.zero_grad()
    out = model(data.x, data.edge_index)
    
    if data.train_mask.any():
        loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
        if torch.isnan(loss) or torch.isinf(loss):
            raise RuntimeError("Loss NaN/Inf; check data")
        loss.backward()
        opt.step()
        loss_val = loss.detach().item()
    else:
        loss_val = float("nan")
    
    if ep % 10 == 0:
        tr_mae, tr_rmse = eval_metrics("train")
        va_mae, va_rmse = eval_metrics("val")
        te_mae, te_rmse = eval_metrics("test")
        print(
            f"ep {ep:03d} | train_loss(MSE) {loss_val:.4f} "
            f"| TR MAE {tr_mae:.3f}y RMSE {tr_rmse:.3f}y "
            f"| VAL MAE {va_mae:.3f}y RMSE {va_rmse:.3f}y "
            f"| TEST MAE {te_mae:.3f}y RMSE {te_rmse:.3f}y"
        )

print("\nTraining completed!")


Starting training...

ep 010 | train_loss(MSE) 0.8799 | TR MAE 0.048y RMSE 0.123y | VAL MAE 0.037y RMSE 0.058y | TEST MAE 0.046y RMSE 0.061y
ep 020 | train_loss(MSE) 0.7422 | TR MAE 0.050y RMSE 0.113y | VAL MAE 0.052y RMSE 0.090y | TEST MAE 0.067y RMSE 0.087y
ep 030 | train_loss(MSE) 0.6695 | TR MAE 0.049y RMSE 0.107y | VAL MAE 0.060y RMSE 0.106y | TEST MAE 0.073y RMSE 0.099y
ep 040 | train_loss(MSE) 0.6303 | TR MAE 0.043y RMSE 0.101y | VAL MAE 0.063y RMSE 0.114y | TEST MAE 0.068y RMSE 0.101y
ep 050 | train_loss(MSE) 0.5297 | TR MAE 0.042y RMSE 0.093y | VAL MAE 0.067y RMSE 0.134y | TEST MAE 0.076y RMSE 0.118y
ep 060 | train_loss(MSE) 0.4291 | TR MAE 0.042y RMSE 0.086y | VAL MAE 0.079y RMSE 0.160y | TEST MAE 0.084y RMSE 0.134y
ep 070 | train_loss(MSE) 0.3527 | TR MAE 0.038y RMSE 0.077y | VAL MAE 0.082y RMSE 0.177y | TEST MAE 0.085y RMSE 0.141y
ep 080 | train_loss(MSE) 0.3524 | TR MAE 0.037y RMSE 0.070y | VAL MAE 0.103y RMSE 0.214y | TEST MAE 0.094y RMSE 0.162y
ep 090 | train_loss(MSE) 

In [46]:

model.eval()
with torch.no_grad():
    final_out = model(data.x, data.edge_index)

tr_mae, tr_rmse = eval_metrics("train")
va_mae, va_rmse = eval_metrics("val")
te_mae, te_rmse = eval_metrics("test")

print("\n" + "="*60)
print("FINAL RESULTS (with Demographics + CSF Biomarkers)")
print("="*60)
print(f"TRAIN | MAE: {tr_mae:.3f} years | RMSE: {tr_rmse:.3f} years")
print(f"VAL   | MAE: {va_mae:.3f} years | RMSE: {va_rmse:.3f} years")
print(f"TEST  | MAE: {te_mae:.3f} years | RMSE: {te_rmse:.3f} years")
print("="*60)

print("\nNOTE: Compare these results with DemographicOnly.ipynb to see")
print("the improvement gained from adding CSF biomarkers!")


FINAL RESULTS (with Demographics + CSF Biomarkers)
TRAIN | MAE: 0.032 years | RMSE: 0.060 years
VAL   | MAE: 0.124 years | RMSE: 0.265 years
TEST  | MAE: 0.103 years | RMSE: 0.181 years

NOTE: Compare these results with DemographicOnly.ipynb to see
the improvement gained from adding CSF biomarkers!


In [47]:

with torch.no_grad():
    h = F.relu(model.c1(data.x, data.edge_index))
    embeddings = h.cpu().numpy()

emb_df = pd.DataFrame(embeddings, index=df.index)
emb_df.insert(0, "RID", df["RID"].values)
emb_df.insert(1, "VISDATE", pd.to_datetime(df[date_col]).dt.date.astype("string"))
emb_df.to_csv("visit_embeddings_with_biomarkers.csv", index=False)

print("\nEmbeddings saved to: visit_embeddings_with_biomarkers.csv")
print(f"Shape: {emb_df.shape}")


Embeddings saved to: visit_embeddings_with_biomarkers.csv
Shape: (6210, 66)


In [48]:

import json

enhanced_results = {
    'model': 'Demographics + CSF Biomarkers',
    'features': data.num_node_features,
    'train_mae': tr_mae,
    'train_rmse': tr_rmse,
    'val_mae': va_mae,
    'val_rmse': va_rmse,
    'test_mae': te_mae,
    'test_rmse': te_rmse
}

with open('enhanced_biomarkers_results.json', 'w') as f:
    json.dump(enhanced_results, f, indent=2)

print("\n✅ Results saved to: enhanced_biomarkers_results.json")
print("\nYou can now run CompareModels.ipynb to compare with baseline!")


✅ Results saved to: enhanced_biomarkers_results.json

You can now run CompareModels.ipynb to compare with baseline!
